In [62]:
ls

Untitled-1.ipynb    cardiovascular.txt:Zone.Identifier  output.csv
cardiovascular.txt  miniconda3/


In [139]:
from pyspark.sql import SparkSession

In [246]:
spark = SparkSession.builder.master('local').appName('PySpark_HW').getOrCreate()

In [247]:
from pyspark.pandas import read_csv


In [248]:

df = spark.read.csv('output.csv', header = True, inferSchema = True)
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- ind: integer (nullable = true)
 |-- sbp: integer (nullable = true)
 |-- tobacco: double (nullable = true)
 |-- ldl: double (nullable = true)
 |-- adiposity: double (nullable = true)
 |-- famhist: string (nullable = true)
 |-- typea: integer (nullable = true)
 |-- obesity: double (nullable = true)
 |-- alcohol: double (nullable = true)
 |-- age: integer (nullable = true)
 |-- chd: integer (nullable = true)



In [249]:
df = df.drop("_c0")


In [250]:
df.show(2)

+---+---+-------+----+---------+-------+-----+-------+-------+---+---+
|ind|sbp|tobacco| ldl|adiposity|famhist|typea|obesity|alcohol|age|chd|
+---+---+-------+----+---------+-------+-----+-------+-------+---+---+
|  1|160|   12.0|5.73|    23.11|Present|   49|   25.3|   97.2| 52|  1|
|  2|144|   0.01|4.41|    28.61| Absent|   55|  28.87|   2.06| 63|  1|
+---+---+-------+----+---------+-------+-----+-------+-------+---+---+
only showing top 2 rows



In [251]:
import pyspark.sql.functions as F

In [252]:
smo = df.filter(F.col('tobacco')<= 0.01).count()
nosmo = df.filter(F.col('tobacco')> 0.01).count()
print(f'c курением в анамнезе - {smo}\n без курения в анамнезе - {nosmo}')

c курением в анамнезе - 108
 без курения в анамнезе - 354


In [253]:
df = df.withColumn("tob/year", F.col('tobacco') / F.col('age'))


In [254]:
df.show(3)

+---+---+-------+----+---------+-------+-----+-------+-------+---+---+--------------------+
|ind|sbp|tobacco| ldl|adiposity|famhist|typea|obesity|alcohol|age|chd|            tob/year|
+---+---+-------+----+---------+-------+-----+-------+-------+---+---+--------------------+
|  1|160|   12.0|5.73|    23.11|Present|   49|   25.3|   97.2| 52|  1| 0.23076923076923078|
|  2|144|   0.01|4.41|    28.61| Absent|   55|  28.87|   2.06| 63|  1|1.587301587301587...|
|  3|118|   0.08|3.48|    32.28|Present|   52|  29.14|   3.81| 46|  0|0.001739130434782...|
+---+---+-------+----+---------+-------+-----+-------+-------+---+---+--------------------+
only showing top 3 rows



In [255]:
from pyspark.ml.feature import StringIndexer, VectorAssembler


In [256]:
label_stringIdx = StringIndexer(inputCol = 'famhist', outputCol = 'famehist2')
df = label_stringIdx.fit(df).transform(df)
df.show(5)

+---+---+-------+----+---------+-------+-----+-------+-------+---+---+--------------------+---------+
|ind|sbp|tobacco| ldl|adiposity|famhist|typea|obesity|alcohol|age|chd|            tob/year|famehist2|
+---+---+-------+----+---------+-------+-----+-------+-------+---+---+--------------------+---------+
|  1|160|   12.0|5.73|    23.11|Present|   49|   25.3|   97.2| 52|  1| 0.23076923076923078|      1.0|
|  2|144|   0.01|4.41|    28.61| Absent|   55|  28.87|   2.06| 63|  1|1.587301587301587...|      0.0|
|  3|118|   0.08|3.48|    32.28|Present|   52|  29.14|   3.81| 46|  0|0.001739130434782...|      1.0|
|  4|170|    7.5|6.41|    38.03|Present|   51|  31.99|  24.26| 58|  1| 0.12931034482758622|      1.0|
|  5|134|   13.6| 3.5|    27.78|Present|   60|  25.99|  57.34| 49|  1| 0.27755102040816326|      1.0|
+---+---+-------+----+---------+-------+-----+-------+-------+---+---+--------------------+---------+
only showing top 5 rows



In [257]:
numericCols = ['sbp', 'tobacco', 'ldl', 'adiposity','famehist2','typea','obesity','alcohol','age','tob/year']
assembler = VectorAssembler(inputCols=numericCols, outputCol="features")
df = assembler.transform(df)
df.show(5)

+---+---+-------+----+---------+-------+-----+-------+-------+---+---+--------------------+---------+--------------------+
|ind|sbp|tobacco| ldl|adiposity|famhist|typea|obesity|alcohol|age|chd|            tob/year|famehist2|            features|
+---+---+-------+----+---------+-------+-----+-------+-------+---+---+--------------------+---------+--------------------+
|  1|160|   12.0|5.73|    23.11|Present|   49|   25.3|   97.2| 52|  1| 0.23076923076923078|      1.0|[160.0,12.0,5.73,...|
|  2|144|   0.01|4.41|    28.61| Absent|   55|  28.87|   2.06| 63|  1|1.587301587301587...|      0.0|[144.0,0.01,4.41,...|
|  3|118|   0.08|3.48|    32.28|Present|   52|  29.14|   3.81| 46|  0|0.001739130434782...|      1.0|[118.0,0.08,3.48,...|
|  4|170|    7.5|6.41|    38.03|Present|   51|  31.99|  24.26| 58|  1| 0.12931034482758622|      1.0|[170.0,7.5,6.41,3...|
|  5|134|   13.6| 3.5|    27.78|Present|   60|  25.99|  57.34| 49|  1| 0.27755102040816326|      1.0|[134.0,13.6,3.5,2...|
+---+---+-------

In [258]:
train, test = df.randomSplit([0.8, 0.2], seed = 20)

In [259]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [260]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'chd')
rfModel = rf.fit(train)
predictions = rfModel.transform(test)
evaluator = MulticlassClassificationEvaluator(labelCol="chd", predictionCol="prediction")
accuracy = evaluator.evaluate(predictions)
print(f'accuracy = {accuracy}%')

accuracy = 0.7016666666666668%
